In [1]:
from align import align, read_fasta
import numpy as np

In [2]:
NW = align.NeedlemanWunsch("./substitution_matrices/BLOSUM62.mat", gap_open = -10, gap_extend = -1)

In [3]:
seq1,_ = read_fasta("./data/test_seq1.fa")
seq2,_ = read_fasta("./data/test_seq2.fa")

In [4]:
NW.align(seq1, seq2)

(4.0, 'MYQR', 'M-QR')

In [5]:
NW._align_matrix

array([[  0., -inf, -inf, -inf],
       [-inf,   5., -11., -13.],
       [-inf, -12.,   4.,  -8.],
       [-inf, -12.,  -1.,   5.],
       [-inf, -14.,  -6.,   4.]])

In [6]:
m = np.array([[0,       -np.inf,    -np.inf,    -np.inf],
                  [-np.inf, 5,          -11,        -13],
                  [-np.inf, -12,        4,         -8],
                  [-np.inf, -12,        -1,         5],
                  [-np.inf, -14,        -6.,        4]])

ALIGNMENT SCORES TO HUMAN BRD2:
text2


In [6]:
sub_dict = NW.sub_dict
sub_dict

{('A', 'A'): 4.0,
 ('R', 'A'): -1.0,
 ('N', 'A'): -2.0,
 ('D', 'A'): -2.0,
 ('C', 'A'): 0.0,
 ('Q', 'A'): -1.0,
 ('E', 'A'): -1.0,
 ('G', 'A'): 0.0,
 ('H', 'A'): -2.0,
 ('I', 'A'): -1.0,
 ('L', 'A'): -1.0,
 ('K', 'A'): -1.0,
 ('M', 'A'): -1.0,
 ('F', 'A'): -2.0,
 ('P', 'A'): -1.0,
 ('S', 'A'): 1.0,
 ('T', 'A'): 0.0,
 ('W', 'A'): -3.0,
 ('Y', 'A'): -2.0,
 ('V', 'A'): 0.0,
 ('B', 'A'): -2.0,
 ('Z', 'A'): -1.0,
 ('X', 'A'): 0.0,
 ('*', 'A'): -4.0,
 ('A', 'R'): -1.0,
 ('R', 'R'): 5.0,
 ('N', 'R'): 0.0,
 ('D', 'R'): -2.0,
 ('C', 'R'): -3.0,
 ('Q', 'R'): 1.0,
 ('E', 'R'): 0.0,
 ('G', 'R'): -2.0,
 ('H', 'R'): 0.0,
 ('I', 'R'): -3.0,
 ('L', 'R'): -2.0,
 ('K', 'R'): 2.0,
 ('M', 'R'): -1.0,
 ('F', 'R'): -3.0,
 ('P', 'R'): -2.0,
 ('S', 'R'): -1.0,
 ('T', 'R'): -1.0,
 ('W', 'R'): -3.0,
 ('Y', 'R'): -2.0,
 ('V', 'R'): -3.0,
 ('B', 'R'): -1.0,
 ('Z', 'R'): 0.0,
 ('X', 'R'): -1.0,
 ('*', 'R'): -4.0,
 ('A', 'N'): -2.0,
 ('R', 'N'): 0.0,
 ('N', 'N'): 6.0,
 ('D', 'N'): 1.0,
 ('C', 'N'): -3.0,
 ('Q', 'N'

In [16]:
seqA = seq1
seqB = seq2
gap_open = -10
gap_extend = - 1




alignment_score = 0

# Initializing sequences for use in backtrace method
_seqA = seqA
_seqB = seqB

# Get sequence lengths; sequences are seq, header
seqA_len = len(seqA[0])
seqB_len = len(seqB[0])

# Initialize matrix private attributes for use in alignment
# create matrices for alignment scores, gaps, and backtracing
_align_matrix = np.full((seqA_len + 1, seqB_len + 1), -np.inf)
_gapA_matrix = np.full((seqA_len + 1, seqB_len + 1), -np.inf)
_gapB_matrix = np.full((seqA_len + 1, seqB_len + 1), -np.inf)
_back = np.full((seqA_len + 1, seqB_len + 1), -np.inf)

# Set values of alignment and gap matrices
_align_matrix[0][0] = 0
for i in range(0, seqA_len + 1):
    _gapA_matrix[i][0] = gap_open + (gap_extend * i)
for j in range(0, seqB_len + 1):
    _gapB_matrix[0][j] = gap_open + (gap_extend * j)


# Loop through and calculate scores
for i in range(1, seqA_len + 1):
    for j in range(1, seqB_len + 1):
        baseA = seqA[0][i-1]
        baseB = seqB[0][j-1]
        m_score = sub_dict[(baseA, baseB)]

        # Fill _align_matrix
        # Take max of m_score + (m, gapA, gapB)
        # When calculaing m, all of them add m_score 
        _m = _align_matrix[i-1][j-1]
        _mIx = _gapA_matrix[i-1][j-1] 
        _mIy = _gapB_matrix[i-1][j-1]
        _align_matrix[i][j] = m_score + max(_m, _mIx, _mIy)
        
        # Fill in _gapA_matrix
        _gapAm = _align_matrix[i][j-1] + gap_open + gap_extend
        _gapAIx = _gapA_matrix[i][j-1] + gap_extend
        _gapAIy = _gapB_matrix[i][j-1] + gap_open + gap_extend 
        _gapA_matrix[i][j] = max(_gapAm, _gapAIx, _gapAIy)

        # Fill in _gapB_matrix
        _gapBm = _align_matrix[i-1][j] + gap_open + gap_extend
        _gapBIx = _gapA_matrix[i-1][j] + gap_open + gap_extend
        _gapBIy = _gapB_matrix[i-1][j] + gap_extend
        _gapB_matrix[i][j] = max(_gapBm, _gapBIy, _gapBIx)

        # Find the best score
        best_score = max(_align_matrix[i][j], _gapA_matrix[i][j], _gapB_matrix[i][j])
        # Annotate source of best score for traceback
        # if best_score from align_matrix, annotate as 0
        # if best_score from _gapA_matrix, annotate as 1
        # if best_score from _gapB_matrix, annotate as 2
        if best_score == _align_matrix[i][j]:
            _back[i][j] = 0
        elif best_score == _gapA_matrix[i][j]:
            _back[i][j] = 1
        else:
            _back[i][j] = 2


In [11]:
seqA = seq1[0]
seqB = seq2[0]

_align_matrix = np.ones((len(seqA) + 1, len(seqB) + 1)) * -np.inf
_gapA_matrix = np.ones((len(seqA) + 1, len(seqB) + 1)) * -np.inf
_gapB_matrix = np.ones((len(seqA) + 1, len(seqB) + 1)) * -np.inf

# create matrix for pointers used in backtrace procedure
_back = np.ones((len(seqA) + 1, len(seqB) + 1)) * -np.inf

# Resetting alignment in case method is called more than once
seqA_align = ""
seqB_align = ""

# Resetting alignment score in case method is called more than once
alignment_score = 0

# Initializing sequences for use in backtrace method
_seqA = seqA
_seqB = seqB

# Initialize algin_matrix
_align_matrix[0][0] = 0

# Initialize gapA_matrix
for i in range(0,len(seqA)+1):
    _gapA_matrix[i][0] = gap_open + (gap_extend * i)

# Initialize gapB_matrix
for j in range(0,len(seqB)+1):
    _gapB_matrix[0][j] = gap_open + (gap_extend * j)

# Loop through matrices
for i in range(1,len(seqA)+1):
    for j in range(1,len(seqB)+1):
        # Get amino acids at position
        baseA = _seqA[i-1]
        baseB = _seqB[j-1]
        m_score = sub_dict[(baseA, baseB)]

        # Fill in align_matrix 
        m = _align_matrix[i-1][j-1] 
        delete = _gapA_matrix[i-1][j-1]
        insert = _gapB_matrix[i-1][j-1]
        _align_matrix[i][j] = m_score + max(m, delete, insert)

        # Fill in gapA_matrix
        gapA_1 = gap_open + gap_extend + _align_matrix[i][j-1]
        gapA_2 = gap_extend + _gapA_matrix[i][j-1]
        gapA_3 = gap_open + gap_extend + _gapB_matrix[i][j-1]
        _gapA_matrix[i][j] = max(gapA_1, gapA_2, gapA_3)

        # Fill in gapB_matrix
        gapB_1 = gap_open + gap_extend + _align_matrix[i-1][j]
        gapB_2 = gap_open + gap_extend + _gapA_matrix[i-1][j]
        gapB_3 = gap_extend + _gapB_matrix[i-1][j]
        _gapB_matrix[i][j] = max(gapB_1, gapB_2, gapB_3)

        # Fill in back matrix for backtracing
        best_score = max(_align_matrix[i][j], _gapA_matrix[i][j], _gapB_matrix[i][j])
        # A match is represented in the back matrix as a 0
        if best_score == _align_matrix[i][j]:
            _back[i][j] = 0
        # A gap in seqA is represented in the back matrix as a -1
        elif best_score == _gapA_matrix[i][j]:
            _back[i][j] = -1
        # A gap in seqB is represented in the back matrix as a 1
        else:
            _back[i][j] = 1        	

In [17]:
_align_matrix

array([[  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf,   5., -11., -10., -12., -15., -17., -18.],
       [-inf, -12.,   4.,  -7.,  -8.,  -9., -11., -11.],
       [-inf, -11.,  -8.,   5.,  -4., -11., -12., -12.],
       [-inf, -15.,  -7., -10.,   2.,  -4.,   1., -10.],
       [-inf, -14.,  -3.,  -9.,  -9.,   3.,  -4.,   0.],
       [-inf, -13., -11.,   1.,  -5., -11.,   0.,  -7.],
       [-inf, -15., -13.,  -8.,   5.,  -8., -11.,  -3.],
       [-inf, -18., -10., -13., -11.,  10.,  -6.,  -9.],
       [-inf, -19., -11., -12., -13.,  -1.,  10.,  -3.],
       [-inf, -21., -14., -14., -14.,  -9.,  -3.,  17.]])

In [12]:
_align_matrix

array([[  0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
       [-inf,   5., -11., -10., -12., -15., -17., -18.],
       [-inf, -12.,   4.,  -7.,  -8.,  -9., -11., -11.],
       [-inf, -11.,  -8.,   5.,  -4., -11., -12., -12.],
       [-inf, -15.,  -7., -10.,   2.,  -4.,   1., -10.],
       [-inf, -14.,  -3.,  -9.,  -9.,   3.,  -4.,   0.],
       [-inf, -13., -11.,   1.,  -5., -11.,   0.,  -7.],
       [-inf, -15., -13.,  -8.,   5.,  -8., -11.,  -3.],
       [-inf, -18., -10., -13., -11.,  10.,  -6.,  -9.],
       [-inf, -19., -11., -12., -13.,  -1.,  10.,  -3.],
       [-inf, -21., -14., -14., -14.,  -9.,  -3.,  17.]])